# Исследование надежности заемщиков.

От кредитного отдела банка получено задание. Требуется выяснить , влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

# 1. Обзор данных.

Первым делом импортируем Pandas, чтобы под рукой были все необходимые инструменты библиотеки.

In [1]:
import  pandas as pd

Сохраняем имеющиеся данные в датафрейм df и выводим первые 10 строк таблицы.

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем на экран общюю информацию о датафрейме:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [5]:
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Мы видим что в таблице 12 столбцов. В двух столбцах тип данных `float64`, в пяти `int64` и еще в пяти `object`.

Согласно имеющейся у нас документации к данным:
* `children` - количество детей в семье
* `days_employed` -  общий трудовой стаж в днях
* `dob_years` - возраст клиента в годах
* `education` - уровень образования клиента
* `education_id` -идентификатор уровня образования
* `family_status` - семейное положение
* `family_status_id` - идентификатор семейного положения
* `gender` - пол клиента
* `income_type` - тип занятости
* `debt - имел` ли задолженность по возврату кредитов
* `total_income` -  ежемесячный доход
* `purpose` - цель получения кредита

В столбцах `days_employed` и `total_income` зачения одинаковые, но не совпадают со значениями в других столбцах, значит в данных есть пропуски.

### Выводы

В таблице, в каждой строке, содержится информация о клиенте. Семейное положение, пол, возраст, трудовой стаж и т.п.. И уже по первым 10 строкам мы видим аномальные значения: 
* в столбце `days_employed` (трудовой стаж в днях), данные имеют отрицательные значения, а у пенсионеров стаж указан в невероятных сотнях тысяч дней.
* в столбце `total_income` тип данных указан как вещественные.
* в столбце `education`, в значениях, строчные буквы встречаются с прописными.
* в столбце `family_status`, в значениях, указаны статусы `женат / замужем` и `гражданский брак`.

Имеющихся данных достаточно для проверки гипотез. Но прежде чем преступить к работе нужно привести данные в порядок: разобраться с пропусками и аномальными значениями.

## 2 Предобработка данных.

### 2.1 Пропущенные значения
Для начала посмотрим есть ли пропущеные значения в нашем датафрейме.

In [6]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах 'days_employed' и 'total_income' пропущено по 2174 значения.

Прежде чем обработать пропуски в этих столбцах, нужно разобраться с аномальными значениями. 

Так в столбце трудовой стаж значения не могут быть отрицательными, а если учесть что стаж указан в целых днях то изначения должны быть целочисленными. 

И средний зароботок заполнен вещественными значениеми, так же приведем столбец со средним зароботком к целочисленным значениям.

Еще раз посмотрим на таблицу, но в этот раз выведем 20 первых строк.

In [7]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Наблюдается закономерность: у людей не вышедших на пенсию стаж имеет отрицательное значение, а у пенсионеров положительное но в невероятных сотнях тысяч дней.

Такого стажа как в нашей таблице, порядка 1000 лет, не может быть даже у самых заслуженных Ветеранов труда. Значит данные указаны в других единицах измерения, скорее всего в часах.

Проведя не сложные вычисления : 340266.072047 / 24 = 14177.753001 (разделим трудовой стаж первого в списке пенсионера на количество часов в сутках)

Мы получили 14177 дней, что уже соответствует примерно 39 годам трудового стажа. Довольно правдоподобные цифры.

Проделав тоже самое для следующего в списке пенсионера (имеющего данные в этом столбце), мы получим примерно 45 лет трудового стажа. Что еще раз подтверждает гипотезу о том что трудовой стаж пенсионеров указан в часах.

Что касательно отрицательных значений в данных столбца `days_employed`.

Проанализировав таблицу можно сказать, отрицательные значения в данных указывают на то что человек еще не вышел на пенсию или не имеет дополнительного дахода в виде социальных выплат.

Так же можно отметить что возраст и стаж не взаимосвязанные еденицы, у некоторых граждан предпенсионного возраста стаж меньше чем у 25 летних, а у других стаж в 53 года соответствует 45 годам. Следовательно пропуски в данных о стаже мы можем заменить медианным значением по столбцу, среднее же значение мы преминить не можем т.к. некоторые значения выделяются над остальным довольно значительно.

То же и касательно среднего дохода, медианное значение более подходящее нежели чем среднее, на фоне разницы доходов пенсионеров и других категорий клиентов.

Посчитаем медианное значение для каждого из столбцов:

In [8]:
df['days_employed'].median()

-1203.369528770489

Для столбца `income_type` заменим пустые значения на медианные согластно типу занятости:

In [9]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

И заменим пропуски:

In [10]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df['total_income'] = df['total_income'].fillna(df.loc[df['income_type'] == 'пенсионер', 'total_income'].median())
df['total_income'] = df['total_income'].fillna(df.loc[df['income_type'] == 'сотрудник', 'total_income'].median())
df['total_income'] = df['total_income'].fillna(df.loc[df['income_type'] == 'компаньон', 'total_income'].median())
df['total_income'] = df['total_income'].fillna(df.loc[df['income_type'] == 'госслужащий', 'total_income'].median())
df['total_income'] = df['total_income'].fillna(df.loc[df['income_type'] == 'безработный', 'total_income'].median())
df['total_income'] = df['total_income'].fillna(df.loc[df['income_type'] == 'предприниматель', 'total_income'].median())
df['total_income'] = df['total_income'].fillna(df.loc[df['income_type'] == 'студент', 'total_income'].median())
df['total_income'] = df['total_income'].fillna(df.loc[df['income_type'] == 'в декрете', 'total_income'].median())

Проверим остались ли пропуски в нашем датафрейме.

In [11]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 2.2 Обработка аномальных значений.

Разберемся с аномальными значениями.

Так в столбце трудовой стаж значения не могут быть отрицательными, а если учесть что стаж указан в целых днях то изначения должны быть целочисленными. И тысячилетний трудовой стаж вернем к правильным значениям.

И средний зароботок заполнен вещественными значениеми, так же приведем столбец со средним зароботком к целочисленным значениям.


In [12]:
df.loc[df['income_type'] == 'пенсионер', 'days_employed'] = df.loc[df['income_type'] == 'пенсионер', 'days_employed'] / 24 # вернем стаж от часов к дням  
df['days_employed'] = df['days_employed'].abs().astype('int') # вернём положительное и целочисленное значение
df['total_income'] = df['total_income'].astype('int') # вернули целочисленное значение и тут

Обновим информацию по нашему датафрейму.

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Теперь значения в интересующих нас столбцах целочисленные.

Посмотрим как теперь выглядит наш датафрейм.

In [14]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Теперь В столбце`days_employed` данные более правдоподобны.

### 2.2 Обработка дубликатов.

#### 2.2.1 Обработка и удаление явных дубликатов.

Сначала посмотрим сколько у нас в таблице явных дубликатов.

In [15]:
df.duplicated().sum() 

54

54 явных дубликата из 21525 строк, значение на уровне погрешности. Удалим явные дубликаты с удалением старых индексов.

In [16]:
df = df.drop_duplicates().reset_index(drop=True)

Ещё раз посчитаем явные дубликаты - что бы убедится в их отсутствии.

In [17]:
df.duplicated().sum()

0

#### 2.2.2 Обработка и устранение неявных дубликатов.

В столбце `education` мы видим что данные об образовании занесены в разном регистре.

Посмотрим сколько уникальных значений в этом столбце:

In [18]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведем все значения к нижнему регистру. Создадим функцию которая нам в этом поможет.

In [19]:
df['education'] = df['education'].str.lower()

Проверяем:

In [20]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Так же проверим другие столбцы на наличие неявных дубликатов:

In [21]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

В столбце семейное положение дубликатов нет, но в статусе 'Не женат / не замужем' первая 'Не' с большой буквы.

In [22]:
df.loc[(df['family_status'] == 'Не женат / не замужем'), 'family_status'] = 'не женат / не замужем' # изменяем статус на нужное нам значение

Еще раз проверим:

In [23]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [24]:
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Тут у нас неявные дубликаты в виде семей с аномально малым количеством детей -1, и самых счастливых семьй с 20 детьми. Скорее всего ошибка была допущена во время ввода данных. Исправим:

In [25]:
df.loc[df['children'] == 20, 'children'] = df.loc[df['children'] == 20, 'children'] / 10 # семьи с 20 детьми бывают, но очень редко, а у нас их 76. Скорее всего 0 после 2 результат ошибки при вводе.
df['children'] = df['children'].abs().astype('int') # уберем отрицательные значения

Проверим:

In [26]:
df['children'].value_counts()

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

С детьми разобрались. Теперь займемся половым вопросом:

In [27]:
df['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

В столбце с указанием пола клиента неожиданный результат - три пола вместо двух (современно конечно, но для нашей работы это не нужно). Ошибка всего в одной строке - удаление этой строки не приведет к сколь либо существенным искажениям.

In [28]:
df = df.loc[df['gender'] != 'XNA'].reset_index(drop=True)

In [29]:
df['gender'].value_counts()

F    14189
M     7281
Name: gender, dtype: int64

#### Выводы по работе с дубликатами.

Для поиска и обработки дубликатов Я использовал стандартные методы duplicated() и drop_duplicates(). Данный метод был выбран исходя из своей простоты и легкости применения.

Для поиска и обработки неявных дубликатов использовал метод unique() и написал собственные функции для их устранения.

Проанализировав таблицу с уверенностью можно сказать, что, в большинстве случаев причина появления дубликатов - человеческий фактор (разный регистр, разные названия одних и тех же категорий и т.п.) и лишь в некоторых случаях ошибки случились по техническим причинам.

### 2.3 Категоризация данных.

Для начала определим как много клиентов имеют задолженность по возврату кредитов.

In [33]:
df['debt'].value_counts()

0    19729
1     1741
Name: debt, dtype: int64

Либо 19729 просроченных платежей и тогда исследование можно окончить: ни кто не латит в срок.

Либо 1741 просроченный платеж, что то же плохо, но с этим можно работать дальше.

Построим и выведем на экран новый датафрейм, в котором проверим есть ли зависимость между образованием и выплатами кредита в срок:

In [34]:
df_education = df.pivot_table(index=['education_id', 'education'], columns='purpose', values='debt', aggfunc=sum, fill_value=0)
df_education

,purpose,автомобили,автомобиль,высшее образование,дополнительное образование,жилье,заняться высшим образованием,заняться образованием,на покупку автомобиля,на покупку подержанного автомобиля,на покупку своего автомобиля,...,профильное образование,ремонт жилью,свадьба,свой автомобиль,сделка с автомобилем,сделка с подержанным автомобилем,строительство жилой недвижимости,строительство недвижимости,строительство собственной недвижимости,сыграть свадьбу
education_id,education,,,,,,,,,,,,,,,,,,,,,
0,высшее,8,6,6,9,4,6,7,8,8,6,...,4,3,9,5,3,14,9,7,10,9
1,среднее,34,35,33,28,39,36,30,31,28,38,...,39,31,52,41,44,34,36,41,31,46
2,неоконченное высшее,2,0,0,1,3,1,1,4,0,1,...,1,1,0,2,1,3,3,5,0,3
3,начальное,0,1,1,0,0,0,1,1,0,1,...,0,0,3,0,2,0,0,1,1,0
4,ученая степень,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Чтож некоторая зависимость присутствует.

А как влияет семейное положение на кредитную историю? Проделаем то же самое и с семейным положением:

In [35]:
df_family = df.pivot_table(index=['family_status_id', 'family_status'], columns='purpose', values='debt', aggfunc='sum', fill_value=0)
df_family

,purpose,автомобили,автомобиль,высшее образование,дополнительное образование,жилье,заняться высшим образованием,заняться образованием,на покупку автомобиля,на покупку подержанного автомобиля,на покупку своего автомобиля,...,профильное образование,ремонт жилью,свадьба,свой автомобиль,сделка с автомобилем,сделка с подержанным автомобилем,строительство жилой недвижимости,строительство недвижимости,строительство собственной недвижимости,сыграть свадьбу
family_status_id,family_status,,,,,,,,,,,,,,,,,,,,,
0,женат / замужем,22,27,25,21,28,25,23,28,20,29,...,27,19,0,20,26,32,33,33,25,0
1,гражданский брак,3,6,8,6,4,6,8,6,5,5,...,9,3,64,7,10,5,5,6,8,58
2,вдовец / вдова,1,1,3,1,1,1,2,2,1,2,...,1,2,0,6,1,4,1,2,0,0
3,в разводе,3,4,0,1,2,4,0,0,3,1,...,2,5,0,3,2,3,2,6,1,0
4,не женат / не замужем,15,4,4,9,11,7,6,8,7,9,...,5,6,0,12,11,7,7,7,8,0


И тут имеется явная закономерность - эти таблицы помогут нам ответить на итоговые вопросы.

Удалим и исходного датафрейма столбцы `education` и `family_status`, так как в новых датафреймах они сапомтвлены с `education_id` и `family_status_id` и по сути дублируют друг друга.

In [36]:
df = df.drop(columns=['education', 'family_status'], axis=1)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


В столбце `total_income` хранятся данные о даходах клиентов в виде целых чисел - это наглядно но неудобно для анализа.

Создадим функцию которая поможет нам разделить клиентов на котегории согласно их доходам:

* 0–30000 — 'E'
* 30001–50000 — 'D'
* 50001–200000 — 'C'
* 200001–1000000 — 'B'
* 1000001 и выше — 'A'

И добавим новый столбец `total_income_category` в наш датафрейм:

In [37]:
def total_income_category(row):
    income = row['total_income']
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'
df['total_income_category'] = df.apply(total_income_category, axis=1)
df['total_income_category'].value_counts()  

C    16033
B     5040
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Как мы можем видеть людей с доходами от 50001 до 200000 больше всего.

In [38]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

А вот тут есть над чем поработать:

* неявные дубликаты
* грамматические ошибки

Выделим основнывные категории заёма и приведём к ним значение в столбце:

* операции с автомобилем
* операции с недвижимостью
* проведение свадьбы
* получение образования

Напишем функцию которая поможет нам внести необходимые изменения и добавим столбец `purpose_category` в наш датафрейм:

In [39]:
def purpose_category(row):
    category = row['purpose']
    if 'авто' in category:
        return 'операции с автомобилем'
    if 'свадьб' in category:
        return 'проведение свадьбы'
    if 'образов' in category:
        return 'получение образования'
    return 'операции с недвижимостью'
df['purpose_category'] = df.apply(purpose_category, axis=1)
df['purpose_category'].value_counts()

операции с недвижимостью    10813
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

Люди берут в долг на операции с недвижимостью гораздо чаще чем на другие цели.

Наш датафрейм обновлен и преведен в порядок, и у нас есть две новые таблицы со сводными данными. Можем попытатся ответить на итоговые вопросы.

## 3 Ответы на вопросы.

### Вопрос №1.

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для того чтобы ответь на этот вопрос - выведем на экран сводную таблицу с данными о количестве детей и просроченных платежах:

In [40]:
df.pivot_table(columns='children', values='debt', aggfunc='sum', fill_value=0)

children,0,1,2,3,4,5
debt,1063,445,202,27,4,0


И тут мы видим что наблюдается прямая зависимость между количеством детей и возвратом кредита в срок. На первый взгляд может показаться что с увеличением количества детей уровень надежности заёмщиков растёт. Но скорее всего это связано с тем что семьи с детьми в целом реже берут кредиты. Построим новую таблицу в которую, помимо количества детей и задолженности, добавим общее количество кредитов в группе и долю должников:

In [41]:
df_family = df.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean'])
df_family = df.groupby(['children']).agg({'debt':['count', 'sum', 'mean']})
df_family.columns = ['кредитов всего', 'просрочка', 'доля должников']
display(df_family)

,кредитов всего,просрочка,доля должников
children,,,
0,14106,1063,0.075358
1,4856,445,0.091639
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


Что и требовалось доказать - чем больше данных тем красочнее картина. 

Теперь мы видим что в группе заемщиков без детей доля должников меньше всего, а в семьях  с 1 и 2 детьми (которых больше всего среди заемщиков с детьми) с увеличением количества детей доля должников только растет. 

Что кастательно семей с 3 детьми наблюдается уменьшение доли просроченных кредитов, возможно это связанно с небольшим количеством кредитов всего в этой группе, и данных для анализа просто не хватает.

Исходя из таблицы самыми надежными являются семьи с 5 детьми, а самыми не надежными семьи с 4 детьми - и это не так. Данных по этим группам слишком мало чтобы мы могли сделать хоть какие либо выводы - поэтому данными по этим группам можно пренебречь.

#### Отвечая на поставленный вопрос , можно утверждать,  наблюдается прямая зависимость: с увеличением количества детей доля просроченных кредитов только растет. Так в группе заемщиков без детей доля должников на 2% меньше чем в семьях с 2 детьми.

### Вопрос №2.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на этот вопрос - выведем на экран сводную таблицу с данными о семейном положении и данными о просроченных платежах:

In [42]:
df.pivot_table(columns='family_status_id', values='debt', aggfunc='sum', fill_value=0)

family_status_id,0,1,2,3,4
debt,931,388,63,85,274


Проверим нет ли здесь той же ошибки что и в предыдущем вопросе:

In [43]:
df_family_status = df.pivot_table(index=['family_status_id'], values='debt', aggfunc=['count', 'sum', 'mean'])
df_family_status = df.groupby(['family_status_id']).agg({'debt':['count', 'sum', 'mean']})
df_family_status.columns = ['кредитов всего', 'просрочка', 'доля должников']
display(df_family_status)

,кредитов всего,просрочка,доля должников
family_status_id,,,
0,12344,931,0.075421
1,4162,388,0.093224
2,959,63,0.065693
3,1195,85,0.071130
4,2810,274,0.097509


Самая маленькая доля должников среди клиентов чей семейный статус `вдовец / вдова`, а самая высокая `не женат / не замужем`.

При этом разница между клиентами со статусом `женат / замужем` и статусом `не женат / не замужем` 2.2%, что может говорить о том что семейные клиенты более надежны чем холостые.

#### Итак, есть ли зависимость между семейным положением и возвратом кредита в срок?  
#### Да некоторая зависимость наблюдается. Клиенты с семейным положением `женат / замужем` одни из самых надежных, а клиенты со статусом `не женат / не замужем` имеют самую большую долю должников.

### Вопрос №3.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для того чтобы ответь на этот вопрос так же построим таблицу и посмотрим на колличество кредитов данных категориях:

In [44]:
df.pivot_table(columns='total_income_category', values='debt', aggfunc='sum', fill_value=0)

total_income_category,A,B,C,D,E
debt,2,356,1360,21,2


In [45]:
df_income = df.pivot_table(index=['total_income_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
df_income = df.groupby(['total_income_category']).agg({'debt':['count', 'sum', 'mean']})
df_income.columns = ['кредитов всего', 'просрочка', 'доля должников']
display(df_income)

,кредитов всего,просрочка,доля должников
total_income_category,,,
A,25,2,0.080000
B,5040,356,0.070635
C,16033,1360,0.084825
D,350,21,0.060000
E,22,2,0.090909


Ввиду малого количества данных о клиентах категорий `A` и `E`, их в расчет брать не будем. 

Что касательно категории `D`, наблюдается самая малая доля должников, но данных для анализа недостаточно.

В категории `C` доля должников самая высокая, на 1.5% выше чем у категории `B`.

#### Ответ: С увеличением дохода  наблидается снижение доли просроченных кредитов.

### Вопрос №4.

####  Как разные цели кредита влияют на его возврат в срок?

Построим сводную таблицу и попробуем сделать вывод:

In [46]:
df.pivot_table(columns='purpose_category', values='debt', aggfunc='sum', fill_value=0)

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
debt,403,782,370,186


In [47]:
df_purpose = df.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
df_purpose = df.groupby(['purpose_category']).agg({'debt':['count', 'sum', 'mean']})
df_purpose.columns = ['кредитов всего', 'просрочка', 'доля должников']
display(df_purpose)

,кредитов всего,просрочка,доля должников
purpose_category,,,
операции с автомобилем,4308,403,0.093547
операции с недвижимостью,10813,782,0.072320
получение образования,4014,370,0.092177
проведение свадьбы,2335,186,0.079657


Самые ненадежные клиенты на катаются на автомобиле, а самые надежные сидят по домам.

#### Разные цели на которые люди берут кредит, по разному влияют на выплату кредита в срок. Так, кредиты на свадбу и на операции с недвижимостью выплачивают  лучше, чем кредиты на автомобиль и образование.

## Выводы.

Мы ответили на 4 поставленных вопроса.

Отвечая на поставленные заказчиком вопросы, выдвинутые гипотизы подтвердились.


Чем можно подытожить проделанную работу:

1. Самыми надежными клиентами являются: бездетные `вдовец / вдова`, с уровнем дохода свыше 200000, для сделок с недвижимостью.

2. Самыми ненадежными клиентами являются: с 2 детьми `не женат / не замужем`, с уровнем дохода от 50000 до 200000, на операции с автомобилем.


Уровень просроченных кредитов не превышает 10%, доля должников варьируется в пределах 2.5% в зависимости от группы, много это или мало решать заказчику. Зависимость от количества детей, семейного положения, уровня дохода и цели кредита прослеживаются. В целом бездетные одинокие клиенты, имеющие даход от 200000 до 1000000 являются более надежными заёмщиками, не взирая на цель кредита. К одиноким клиентам с детьми, берущим кредит на автомобиль, банку нужно быть внимательнее - ибо данная категория заёмщиков является самой ненадежной.
